<a href="https://colab.research.google.com/github/Baraa710/Premier_League_Predictor/blob/main/Results_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler




In [ ]:
# Read the Excel file
df = pd.read_excel("/content/drive/MyDrive/APS360_Project/Match_Data/FinalMatchData.xlsx")

In [ ]:
print(df.shape)

(6459, 248)


In [ ]:
print(df.iloc[:,:])

      Unnamed: 0  Wk  winner  home wins  away wins  home games  away games  \
0              0   1       0          0          0           0           0   
1              1   1       0          0          0           0           0   
2              2   1       0          0          0           0           0   
3              3   1       2          0          0           0           0   
4              4   1       0          0          0           0           0   
...          ...  ..     ...        ...        ...         ...         ...   
6454         375  38       0          7         11          37          37   
6455         376  38       0          8         11          37          37   
6456         377  38       0         17         18          37          37   
6457         378  38       1          7         17          37          37   
6458         379  38       0         14         28          37          37   

      predicted_winner  Age_home_player_2  MP_home_player_2  ..

In [ ]:
# Assuming 'winner_indices' is the column name for labels
labels = df['winner'].values

# Drop the label column to get features
features = df.drop(columns=['winner','Unnamed: 0']).values

# Instantiate the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the features
normalized_features = scaler.fit_transform(features)

# Convert to PyTorch tensors
features_tensor = torch.tensor(normalized_features, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

features_tensor[torch.isnan(features_tensor)] = 0

In [ ]:
print(features_tensor)
print(features_tensor.shape)

tensor([[0.0000, 0.3889, 0.3889,  ..., 0.3077, 0.3091, 1.0000],
        [0.0000, 0.1667, 0.3333,  ..., 0.6923, 0.8636, 1.0000],
        [0.0000, 0.1111, 0.1667,  ..., 0.2308, 0.0727, 1.0000],
        ...,
        [1.0000, 0.6111, 0.1111,  ..., 1.0000, 0.9182, 0.0000],
        [1.0000, 0.5000, 0.2778,  ..., 0.0000, 0.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 0.3846, 0.8545, 1.0000]])
torch.Size([150, 246])


In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_features, rem_features, train_labels, rem_labels = train_test_split(features_tensor, labels_tensor, test_size=0.3, random_state=50)

# Further split the training set into training and validation sets
val_features, test_features, val_labels, test_labels = train_test_split(rem_features, rem_labels, test_size=0.6, random_state=50)

train_dataset = TensorDataset(train_features, train_labels)
val_dataset = TensorDataset(val_features, val_labels)
test_dataset = TensorDataset(test_features, test_labels)

In [ ]:
# prompt: modify get_accuracy(model, batch_size = 64, train=False, test = False): so that it takes another argument, certainty, and calculate accuracy for outputs with given certainty

def get_accuracy(model, batch_size = 64, train=False, test = False, certainty=0.0):
    if train:
        data = train_dataset
    elif test:
        data = test_dataset
    else:
        data = val_dataset

    correct = 0
    total = 0
    for imgs, labels in torch.utils.data.DataLoader(data, batch_size=batch_size):
        model.eval()
        output = model(imgs)
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        #filter out predictions with low certainty
        confident_predictions = pred[output.max(1)[0] > certainty]
        confident_labels = labels[output.max(1)[0] > certainty]
        correct += confident_predictions.eq(confident_labels.view_as(confident_predictions)).sum().item()
        total += confident_labels.shape[0]
    return correct / total


In [ ]:
def get_accuracy(model, batch_size=64, train=False, test=False, certainty=0.0):
    if train:
        data = train_dataset
    elif test:
        data = test_dataset
    else:
        data = val_dataset

    correct = 0
    total = 0
    confident_matches = 0
    num_classes = 3
    class_counts = [0] * num_classes  # Assuming num_classes is defined elsewhere

    for imgs, labels in torch.utils.data.DataLoader(data, batch_size=batch_size):
        print(len(imgs[0]))
        output = model(imgs)
        # Select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        # Filter out predictions with low certainty
        confident_predictions = pred[output.max(1)[0] > certainty]
        confident_labels = labels[output.max(1)[0] > certainty]
        correct += confident_predictions.eq(confident_labels.view_as(confident_predictions)).sum().item()
        total += confident_labels.shape[0]
        confident_matches += confident_labels.shape[0]

        # Count predictions for each class
        for p in confident_predictions:
            class_counts[p.item()] += 1

    # Calculate accuracy and percentage of predictions in each class
    accuracy = correct / total
    class_percentages = [count / confident_matches * 100 for count in class_counts]

    return accuracy, confident_matches, class_percentages


In [ ]:
model = torch.jit.load('/content/drive/MyDrive/APS360_Project/Models/final_model_scripted.pt')
model.eval()


RecursiveScriptModule(
  original_name=NeuralNetwork
  (fc1): RecursiveScriptModule(original_name=Linear)
  (fc2): RecursiveScriptModule(original_name=Linear)
  (fc3): RecursiveScriptModule(original_name=Linear)
  (fc4): RecursiveScriptModule(original_name=Linear)
  (fc5): RecursiveScriptModule(original_name=Linear)
  (fc6): RecursiveScriptModule(original_name=Linear)
  (fc7): RecursiveScriptModule(original_name=Linear)
  (fc8): RecursiveScriptModule(original_name=Linear)
  (fc9): RecursiveScriptModule(original_name=Linear)
  (fc10): RecursiveScriptModule(original_name=Linear)
)

In [ ]:
get_accuracy(model=model, test = True, certainty = 0)

246
246
246
246
246
246
246
246
246
246
246
246
246
246
246
246
246
246
246


(0.524505588993981, 1163, [60.533104041272566, 39.46689595872743, 0.0])

In [ ]:
get_accuracy(model=model, test = True, certainty = 0.1)

(0.524505588993981, 1163, [60.533104041272566, 39.46689595872743, 0.0])

In [ ]:
get_accuracy(model=model, test = True, certainty = 0.3)

(0.5757225433526012, 865, [65.20231213872833, 34.797687861271676, 0.0])

In [ ]:
get_accuracy(model=model, test = True, certainty = 0.5)

(0.617816091954023, 696, [62.643678160919535, 37.35632183908046, 0.0])

In [ ]:
get_accuracy(model=model, test = True, certainty = 0.7)

(0.6436170212765957, 564, [59.57446808510638, 40.42553191489361, 0.0])

In [ ]:
get_accuracy(model=model, test = True, certainty = 0.9)

(0.684331797235023, 434, [57.373271889400925, 42.626728110599075, 0.0])

In [ ]:
for imgs, labels in torch.utils.data.DataLoader(val_dataset):
        print(len(imgs))
        output = model(imgs)
        print(output)

1
tensor([[0.1642, 0.0982, 0.0169]], grad_fn=<DifferentiableGraphBackward>)
1
tensor([[ 1.9855, -1.3548, -0.3116]], grad_fn=<DifferentiableGraphBackward>)
1
tensor([[-0.0635,  0.3271,  0.0244]], grad_fn=<DifferentiableGraphBackward>)
1
tensor([[-0.7973,  1.5476, -0.1339]], grad_fn=<DifferentiableGraphBackward>)
1
tensor([[ 1.2238, -0.7994, -0.1721]], grad_fn=<DifferentiableGraphBackward>)
1
tensor([[0.1875, 0.0787, 0.0133]], grad_fn=<DifferentiableGraphBackward>)
1
tensor([[0.1268, 0.1295, 0.0218]], grad_fn=<DifferentiableGraphBackward>)
1
tensor([[-0.4465,  0.9069, -0.0606]], grad_fn=<DifferentiableGraphBackward>)
1
tensor([[ 1.6646, -1.1187, -0.2536]], grad_fn=<DifferentiableGraphBackward>)
1
tensor([[-0.3994,  0.8251, -0.0473]], grad_fn=<DifferentiableGraphBackward>)
1
tensor([[-0.6043,  1.1948, -0.0975]], grad_fn=<DifferentiableGraphBackward>)
1
tensor([[0.2519, 0.0238, 0.0021]], grad_fn=<DifferentiableGraphBackward>)
1
tensor([[ 0.4514, -0.1460, -0.0314]], grad_fn=<DifferentiableG

In [ ]:
fixture_data = pd.read_excel('/content/drive/MyDrive/APS360_Project/Match_Data/Fixtures_With_Players/2023-2024.xlsx').drop(columns=['Unnamed: 0'])

In [ ]:
fixture_data = fixture_data.drop(columns=[col for col in fixture_data.columns if col.endswith('-url')])


In [ ]:
squad_data_df = pd.read_excel('/content/drive/MyDrive/APS360_Project/Squad_data/squad_data_stats.xls',sheet_name='2022-2023')
player_data_df = pd.read_excel('/content/drive/MyDrive/APS360_Project/Player_data/player_data_cleaned/2022-2023.xlsx',sheet_name='Players').drop(columns = ['Unnamed: 0'])
player_data_df = player_data_df.fillna(0)
squad_standings_df = pd.read_excel('/content/drive/MyDrive/APS360_Project/Squad_data/standings_dataframes.xlsx',sheet_name='2022-2023').drop(columns = ['Unnamed: 0'])

In [ ]:
player_data_df = player_data_df.groupby('Player').agg({'Age': 'max', 'MP': 'sum', 'Min': 'sum', 'Gls': 'sum', 'Ast': 'sum', 'PK': 'sum', 'PKatt': 'sum', 'CrdY': 'sum', 'CrdR': 'sum', 'Starts': 'sum'})

In [ ]:
squad_data_df = pd.read_excel('/content/drive/MyDrive/APS360_Project/Squad_data/squad_data_stats.xls',sheet_name='2022-2023')
squad_data_df = squad_standings_df.merge(squad_data_df, on = 'Squad', how='left')

In [ ]:
fixture_data_2 = pd.read_excel('/content/drive/MyDrive/APS360_Project/Match_Data/cleaned_fixture_data.xlsx', sheet_name='2023-2024').drop(columns=['Unnamed: 0'])

In [ ]:
fixture_data = fixture_data.merge(fixture_data_2, on=['Home','Away'], how='right')

In [ ]:
fixture_data = fixture_data.merge(squad_data_df, left_on=['Home'], right_on='Squad', how='left')

In [ ]:
fixture_data = fixture_data.merge(squad_data_df, left_on=['Away'], right_on='Squad', how='left', suffixes=['_home','_away'])

In [ ]:
home_player_cols = [col for col in fixture_data.columns if col.startswith('Home-Player')]
away_player_cols = [col for col in fixture_data.columns if col.startswith('Away-Player')]

In [ ]:
for i, player_col in enumerate(home_player_cols):
  if i==0:
    continue
  fixture_data = fixture_data.merge(player_data_df, left_on=[player_col], right_on='Player', how='left', suffixes = ['',f'_home_player_{i+1}'])

for i, player_col in enumerate(away_player_cols):
  if i==0:
    continue
  fixture_data = fixture_data.merge(player_data_df, left_on=[player_col], right_on='Player', how='left', suffixes = ['',f'_away_player_{i+1}'])


In [ ]:
fixture_data.drop(columns=home_player_cols+away_player_cols, inplace= True)

In [ ]:

fixture_data.rename(columns={'Age':'Age_home_player_2',
 'MP':'MP_home_player_2',
 'Min':'Min_home_player_2',
 'Gls':'Gls_home_player_2',
 'Ast':'Ast_home_player_2',
 'PK':'PK_home_player_2',
 'PKatt':'PKatt_home_player_2',
 'CrdY':'CrdY_home_player_2',
 'CrdR':'CrdR_home_player_2',
 'Starts':'Starts_home_player_2'}, inplace=True)


In [ ]:

fixture_data.drop(columns=['Home', 'Away','Score'], inplace = True)

In [ ]:
fixture_data.drop(columns=['Squad_away', 'Squad_home'], inplace = True)

In [ ]:

import pandas as pd

# Read the Excel file
df = pd.read_excel("/content/drive/MyDrive/APS360_Project/Match_Data/FinalMatchData.xlsx")




In [ ]:
df.drop(columns=['Unnamed: 0'],inplace=True)


In [ ]:
#  rearrange columns to fit out model input format
column_names = list(df.columns)
fixture_data = fixture_data[column_names]

In [ ]:

fixture_data = fixture_data[-150:]


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Assuming 'winner_indices' is the column name for labels
labels = fixture_data['winner'].values

# Drop the label column to get features
features = fixture_data.drop(columns=['winner']).values

# Instantiate the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the features
normalized_features = scaler.fit_transform(features)

# Convert to PyTorch tensors
features_tensor = torch.tensor(normalized_features, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

features_tensor[torch.isnan(features_tensor)] = 0

In [ ]:
test_dataset = TensorDataset(features_tensor, labels_tensor)

In [ ]:
# prompt: get accuracy of /content/drive/MyDrive/APS360_Project/Models/final_model_scripted.pt on test_dataset

model = torch.jit.load('/content/drive/MyDrive/APS360_Project/Models/final_model_scripted.pt')
model.eval()
get_accuracy(model=model, test = True, certainty = 0)


246
246
246


(0.5, 150, [84.66666666666667, 15.333333333333332, 0.0])

In [ ]:
get_accuracy(model=model, test = True, certainty = 0.1)

246
246
246


(0.5, 150, [84.66666666666667, 15.333333333333332, 0.0])

In [ ]:
get_accuracy(model=model, test = True, certainty = 0.3)

246
246
246


(0.5223880597014925, 134, [87.31343283582089, 12.686567164179104, 0.0])

In [ ]:
get_accuracy(model=model, test = True, certainty = 0.5)

246
246
246


(0.525, 120, [86.66666666666667, 13.333333333333334, 0.0])

In [ ]:
get_accuracy(model=model, test = True, certainty = 0.7)

246
246
246


(0.5370370370370371, 108, [87.03703703703704, 12.962962962962962, 0.0])

In [ ]:
get_accuracy(model=model, test = True, certainty = 0.9)

246
246
246


(0.5555555555555556, 99, [87.87878787878788, 12.121212121212121, 0.0])

In [ ]:
fixture_data = pd.read_excel('/content/drive/MyDrive/APS360_Project/Match_Data/Fixtures_With_Players/2023-2024.xlsx').drop(columns=['Unnamed: 0'])

In [ ]:
fixture_data = fixture_data.drop(columns=[col for col in fixture_data.columns if col.endswith('-url')])


In [ ]:
squad_data_df = pd.read_excel('/content/drive/MyDrive/APS360_Project/Squad_data/squad_data_stats.xls',sheet_name='2022-2023')
player_data_df = pd.read_excel('/content/drive/MyDrive/APS360_Project/Player_data/player_data_cleaned/2022-2023.xlsx',sheet_name='Players').drop(columns = ['Unnamed: 0'])
player_data_df = player_data_df.fillna(0)
squad_standings_df = pd.read_excel('/content/drive/MyDrive/APS360_Project/Squad_data/standings_dataframes.xlsx',sheet_name='2022-2023').drop(columns = ['Unnamed: 0'])

In [ ]:
player_data_df = player_data_df.groupby('Player').agg({'Age': 'max', 'MP': 'sum', 'Min': 'sum', 'Gls': 'sum', 'Ast': 'sum', 'PK': 'sum', 'PKatt': 'sum', 'CrdY': 'sum', 'CrdR': 'sum', 'Starts': 'sum'})

In [ ]:
squad_data_df = pd.read_excel('/content/drive/MyDrive/APS360_Project/Squad_data/squad_data_stats.xls',sheet_name='2022-2023')
squad_data_df = squad_standings_df.merge(squad_data_df, on = 'Squad', how='left')

In [ ]:
fixture_data_2 = pd.read_excel('/content/drive/MyDrive/APS360_Project/Match_Data/cleaned_fixture_data.xlsx', sheet_name='2023-2024').drop(columns=['Unnamed: 0'])

In [ ]:
fixture_data = fixture_data.merge(fixture_data_2, on=['Home','Away'], how='right')

In [ ]:
fixture_data = fixture_data.merge(squad_data_df, left_on=['Home'], right_on='Squad', how='left')

In [ ]:
fixture_data = fixture_data.merge(squad_data_df, left_on=['Away'], right_on='Squad', how='left', suffixes=['_home','_away'])

In [ ]:
home_player_cols = [col for col in fixture_data.columns if col.startswith('Home-Player')]
away_player_cols = [col for col in fixture_data.columns if col.startswith('Away-Player')]

In [ ]:
for i, player_col in enumerate(home_player_cols):
  if i==0:
    continue
  fixture_data = fixture_data.merge(player_data_df, left_on=[player_col], right_on='Player', how='left', suffixes = ['',f'_home_player_{i+1}'])

for i, player_col in enumerate(away_player_cols):
  if i==0:
    continue
  fixture_data = fixture_data.merge(player_data_df, left_on=[player_col], right_on='Player', how='left', suffixes = ['',f'_away_player_{i+1}'])


In [ ]:
fixture_data.drop(columns=home_player_cols+away_player_cols, inplace= True)

In [ ]:

fixture_data.rename(columns={'Age':'Age_home_player_2',
 'MP':'MP_home_player_2',
 'Min':'Min_home_player_2',
 'Gls':'Gls_home_player_2',
 'Ast':'Ast_home_player_2',
 'PK':'PK_home_player_2',
 'PKatt':'PKatt_home_player_2',
 'CrdY':'CrdY_home_player_2',
 'CrdR':'CrdR_home_player_2',
 'Starts':'Starts_home_player_2'}, inplace=True)


In [ ]:

fixture_data.drop(columns=['Home', 'Away','Score'], inplace = True)

In [ ]:
fixture_data.drop(columns=['Squad_away', 'Squad_home'], inplace = True)

In [ ]:

import pandas as pd

# Read the Excel file
df = pd.read_excel("/content/drive/MyDrive/APS360_Project/Match_Data/FinalMatchData.xlsx")




In [ ]:
df.drop(columns=['Unnamed: 0'],inplace=True)


KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
#  rearrange columns to fit out model input format
column_names = list(df.columns)
fixture_data = fixture_data[column_names]

In [ ]:

fixture_data_partial = fixture_data[:50]


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Assuming 'winner_indices' is the column name for labels
labels = fixture_data_partial['winner'].values

# Drop the label column to get features
features = fixture_data_partial.drop(columns=['winner']).values

# Instantiate the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the features
normalized_features = scaler.fit_transform(features)

# Convert to PyTorch tensors
features_tensor = torch.tensor(normalized_features, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

features_tensor[torch.isnan(features_tensor)] = 0

In [ ]:
test_dataset = TensorDataset(features_tensor, labels_tensor)

In [ ]:
# prompt: get accuracy of /content/drive/MyDrive/APS360_Project/Models/final_model_scripted.pt on test_dataset

model = torch.jit.load('/content/drive/MyDrive/APS360_Project/Models/final_model_scripted.pt')
model.eval()
get_accuracy(model=model, test = True, certainty = 0)


246


(0.48, 50, [92.0, 8.0, 0.0])

In [ ]:
get_accuracy(model=model, test = True, certainty = 0.1)

246


(0.48, 50, [92.0, 8.0, 0.0])

In [ ]:
get_accuracy(model=model, test = True, certainty = 0.3)

246


(0.4888888888888889, 45, [97.77777777777777, 2.2222222222222223, 0.0])

In [ ]:
get_accuracy(model=model, test = True, certainty = 0.5)

246


(0.5238095238095238, 42, [97.61904761904762, 2.380952380952381, 0.0])

In [ ]:
get_accuracy(model=model, test = True, certainty = 0.7)

246


(0.5365853658536586, 41, [97.5609756097561, 2.4390243902439024, 0.0])

In [ ]:
get_accuracy(model=model, test = True, certainty = 0.9)

246
246
246
246


(0.5364238410596026, 151, [87.41721854304636, 12.582781456953644, 0.0])